In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import all the necessary libraries and tell why 
from utils.Configs import * #has the parameters for the various models
from tensorflow.python.keras import backend as K
#provides low-level operations for Keras models, such as: Tensor manipulations Mathematical operations Session management Device configuration (e.g., CPU/GPU selection) Memory handling
# from cnn.inputpipeline import InputProcessor
# from cnn.mammocnn import MammoCNN
# from utils.SavedResult import ProcessResult
# from cnn.binaryoptimzers import feature_optimizer


2025-04-23 05:12:46.255235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#we need to have the data set by converting images to image  array
#details needed about the image dataset
data_params={
        'num_classes':len(mammo_mias_classes), 'class_names':mammo_mias_classes, 
        'input_dataset':mias_input_dataset, 'testing_dataset':mias_input_dataset,
        'img_size':mammo_img_size,  'num_channels':mammo_num_channels,
        'train_split':train_split, 'test_split':test_split, 'eval_split':eval_split,
        'train_using':train_using_mammo_mias, 'K':K
        }

In [ ]:
#we use this to fetech images and store them in the array  and split them to train , test and evaluation
from cnn.inputpipeline import InputProcessor
mammo_input_mias=InputProcessor(data_params=data_params) 
mammo_input_mias.get_train_input()

N : 4 : 3 : 0 : 0
BC : 5 : 3 : 0 : 0
BM : 3 : 2 : 0 : 0
CALC : 13 : 9 : 1 : 1
M : 5 : 3 : 0 : 0
finally this is what we have as our training, eval and test
20
1
1


In [5]:

xtrain, ytrain=mammo_input_mias.get_training_data()
xeval, yeval=mammo_input_mias.get_eval_data()
x_test, y_test=mammo_input_mias.get_test_data()


In [6]:
#now we have to build a model that can extract the features form the image dataset
#parameters for the class which builds this model
'''
Run1:optimizers[6], learning_rates[5]
Run2:optimizers[5], learning_rates[5]
Run3:optimizers[1], learning_rates[5]
Run4:optimizers[1], learning_rates[4] 
'''

opt, lr=optimizers[1], learning_rates[4]
mamocnn_paras={
    'learning_rates':lr,  'optimizers':opt, 'activations':activations[1], 
    'pooling':pooling[1], 'regularizers':regularizers[1], 'fcactivations':fcactivations[0], 
    'lossfunc':lossfunc[0], 'cnn_epoch':cnn_epoch, 'batch_size':batch_size,        
    'log_mode':log_mode, 'models_path':models_path, 'model_filename':mammo_model_filename,
    'save_results_dir':save_results_dir, 'show':show, 'K':K, 
    'checkpoint_epoch':40, 'fromCheckpoint':False, 'train_model': True,
    
    'num_classes':len(mammo_mias_classes), 'class_names':mammo_mias_classes, 
    'img_size':mammo_img_size, 'num_channels':mammo_num_channels, 
    'input_source':mammo_input_mias,
    "experiment":"mammocnn_eph{}_optz{}_lr{}".format(cnn_epoch,opt,lr),
    "filename":"mammocnn_eph{}_optz{}_lr{}".format(cnn_epoch,opt,lr),
    'optimzed_result_file':"optimized_mammo_eph{}_{}_lr{}".format(cnn_epoch,opt,lr),
    
    'checkpoint_path':mammo_checkpoint_path, "save_multimodal_results_dir":save_mammo_results_dir,         
}

In [7]:
#we build the model architecture for the that extracts features from the images 
from cnn.mammocnn import MammoCNN
mcnn=MammoCNN(mamocnn_paras=mamocnn_paras)
mcnn.build_architecture()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/reshaping/zero_padding2d.py:72: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: None, Rate: 0.0002
Regularizer type: L1, Rate: 0.0002


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d (ZeroPadding2D)  │ (None, 301, 301, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_1 (Conv2D)               │ (None, 301, 301, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_2 (Conv2D)               │ (None, 301, 301, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (AveragePooling2D)       │ (None, 151, 151, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2_1 (Conv2D)               │ (None, 151, 151, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2_2 (Conv2D)               │ (None, 151, 151, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (AveragePooling2D)       │ (None, 76, 76, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3_1 (Conv2D)               │ (None, 76, 76, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3_2 (Conv2D)               │ (None, 76, 76, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_3 (AveragePooling2D)       │ (None, 38, 38, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4_1 (Conv2D)               │ (None, 38, 38, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_4_2 (Conv2D)               │ (None, 38, 38, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_4 (AveragePooling2D)       │ (None, 19, 19, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 92416)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 92416)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 92416)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ loss_classifier_0 (Dense)       │ (None, 5)              │       462,085 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ class_prob (Activation)         │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,633,765 (6.23 MB)

 Trainable params: 1,633,765 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# now we train the model which will extract the features 
if mamocnn_paras['train_model']:
    if mamocnn_paras['fromCheckpoint']:
        training_outcome=mcnn.load_trained_model(xtrain, ytrain, xtrain, ytrain)
    else:
        training_outcome=mcnn.trainmodel(mcnn.model, xtrain, ytrain, xtrain, ytrain)

    mcnn.extract_vector_features(mcnn.model, xtrain, ytrain, xtrain, ytrain, 'pool_4')
else:
    mcnn.extract_vector_features(mcnn.model, xtrain, ytrain, xtrain, ytrain, 'pool_4')                                                                                                                   

(20, 299, 299, 1) (20, 5)
test loss, test acc: {}
Evaluate on test data


/opt/anaconda3/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
Train features shape: (20, 92416)
Test features shape: (20, 92416)


In [11]:
import pandas as pd
from IPython.display import display
# pd.set_option('display.float_format', '{:.10f}'.format)

In [21]:
#this are extracted features from the image datasets above
lb, ub, x, y, tx, ty = mcnn.optimize_vector_features(xtrain, ytrain, xtrain, ytrain)
print("x: ", x)

selected_x = pd.DataFrame(x)
selected_ts = pd.DataFrame(tx)

# Display only the first 20 columns
display(selected_x.iloc[:, :100])
# display(selected_ts.iloc[:, :20])

x:  [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -7.9313827e-07
  -1.2014332e-06  2.8188756e-06]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -7.9313827e-07
  -1.2014332e-06  2.8188756e-06]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
4,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
5,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
6,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
7,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
8,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
9,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,...,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000


In [37]:
# now lets use hbos algorithms to optimise and get the best features from this entire features 
from cnn.binaryoptimzers import feature_optimizer
from utils.SavedResult import ProcessResult

pr=ProcessResult(params=mamocnn_paras)

#use this path and store the optimised features
save_optimized_path=mamocnn_paras['checkpoint_path']

#Send features to binary optimizer for optimization process
#We are using the BEOSA method for the optimization, hence the selection of first binaryalgorithms[0]
method = binaryalgorithms[0] #for method in binaryalgorithms:
new_x_train, y_train=feature_optimizer(save_optimized_path, method, mcnn.model, x, y, tx, ty, modelrates, lb, ub, pr, mamocnn_paras['num_classes'], 'mammo', metrics_dir) 

================== Population Size ======================
20
about to  start the hbeosa
evolution will start soon
ebola evolution has started
featurebinarization
[(None, [array([0., 0., 0., ..., 0., 0., 0.]), 0.1485025969529086]), (17, [array([0., 0., 0., ..., 0., 0., 0.]), 0.14881801852493076]), (16, [array([0., 0., 0., ..., 0., 0., 0.]), 0.14893574705678672]), (0, [array([0., 0., 0., ..., 0., 0., 0.]), 0.1491704466759003]), (15, [array([0., 0., 0., ..., 0., 0., 0.]), 0.14920777787396125]), (2, [array([0., 0., 0., ..., 0., 0., 0.]), 0.1493266966759003]), (8, [array([0., 0., 0., ..., 1., 0., 0.]), 0.15005503376038784]), (7, [array([1., 0., 0., ..., 0., 0., 0.]), 0.1502482903393352]), (5, [array([0., 0., 1., ..., 0., 0., 0.]), 0.1502873528393352]), (10, [array([0., 0., 0., ..., 1., 1., 0.]), 0.15036742555401664]), (18, [array([0., 0., 0., ..., 0., 0., 0.]), 0.15046600155817177]), (19, [array([0., 0., 0., ..., 0., 0., 0.]), 0.15070394736842108]), (6, [array([0., 0., 0., ..., 0., 0., 0.])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
2,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
3,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000
4,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
5,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
6,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
7,1.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
8,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000
9,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,1.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000


,Fitness
0,0.1485025970
1,0.1488180185
2,0.1489357471
3,0.1491704467
4,0.1492077779
5,0.1493266967
6,0.1500550338
7,0.1502482903
8,0.1502873528
9,0.1503674256


face  0  of the pandimec


/Users/justice/Projects/CNN-v1/utils/differential_equations.py:52: RuntimeWarning: invalid value encountered in divide
  self.ys=(self.ys - np.min(self.ys)) / (np.max(self.ys) - np.min(self.ys))
/Users/justice/Projects/CNN-v1/utils/differential_equations.py:52: RuntimeWarning: invalid value encountered in divide
  self.ys=(self.ys - np.min(self.ys)) / (np.max(self.ys) - np.min(self.ys))


current best fit ==  0.14660715026904622
Iteration  0 : Best Fit =  0.14660715026904622
[(14, [array([ 0.        , -0.57511853,  0.        , ...,  0.        ,
        0.        ,  0.        ]), 0.14660715026904622]), (13, [array([0., 0., 0., ..., 0., 0., 0.]), 0.1471094637588545]), (6, [array([0., 0., 0., ..., 0., 0., 0.]), 0.14732860737555156]), (10, [array([ 0.        ,  0.        ,  0.        , ..., -0.42923785,
       -0.42923785,  0.        ]), 0.14770214194219286]), (8, [array([ 0.        ,  0.        ,  0.        , ..., -0.21601986,
        0.        ,  0.        ]), 0.14816723976904828]), (5, [array([ 0.        ,  0.        , -0.03543379, ...,  0.        ,
        0.        ,  0.        ]), 0.14843935628749946]), (None, [array([0., 0., 0., ..., 0., 0., 0.]), 0.1485025969529086]), (12, [array([0.00462491, 0.00462491, 0.00462491, ..., 0.        , 0.        ,
       0.        ]), 0.14852551330397487]), (16, [array([0., 0., 0., ..., 0., 0., 0.]), 0.14868965261669284]), (0, [array([

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0000000000,-0.5751185263,0.0000000000,0.0000000000,-0.5751185263,0.0000000000,-0.5751185263,0.0000000000,-0.5751185263,-0.5751185263,-0.5751185263,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,0.0000000000,0.0000000000,0.0000000000,-0.4485517098,0.0000000000,0.0000000000,-0.4485517098,-0.4485517098,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-0.4485517098,0.0000000000,0.0000000000
2,0.0000000000,0.0000000000,0.0000000000,-0.4830176333,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-0.4830176333,-0.4830176333,0.0000000000,0.0000000000,0.0000000000,-0.4830176333,-0.4830176333,0.0000000000,0.0000000000,0.0000000000,-0.4830176333,0.0000000000
3,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-0.4292378454,0.0000000000,-0.4292378454,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-0.4292378454,0.0000000000,0.0000000000,-0.4292378454,0.0000000000,0.0000000000,0.0000000000,0.0000000000
4,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-0.2160198613,0.0000000000,-0.2160198613,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
5,0.0000000000,0.0000000000,-0.0354337901,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,-0.0354337901,0.0000000000,0.0000000000,0.0000000000
6,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
7,0.0046249072,0.0046249072,0.0046249072,0.0046249072,0.0046249072,0.0046249072,0.0046249072,0.0046249072,0.0000000000,0.0000000000,0.0046249072,0.0000000000,0.0000000000,0.0000000000,0.0046249072,0.0046249072,0.0000000000,0.0046249072,0.0046249072,0.0046249072
8,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.4318495185,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
9,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.2819363524


,Fitness
0,0.1466071503
1,0.1471094638
2,0.1473286074
3,0.1477021419
4,0.1481672398
5,0.1484393563
6,0.1485025970
7,0.1485255133
8,0.1486896526
9,0.1486908881


================== Transforming Features ======================


In [16]:
print(new_x_train.shape)
display(new_x_train)

(20, 92416)


array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -7.9313827e-07, -1.2014332e-06,  2.8188756e-06],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -7.9313827e-07, -1.2014332e-06,  2.8188756e-06],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)